# Récupération des données de mobilité humaines de Apple

> Ce Notebook présente les données de mobilités humaines depuis le début de la pandémie.

> Les données de mobilités humaines sont fournies par Apple et accessibles à l'adresse suivante : https://covid19.apple.com/mobility. Les données de mobilités commencent à partir du **13 Janvier 2020**. La référence (0%) est faite sur la valeur médiane des données du **13 Janvier 2020**.

## Importation des modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from math import *
from seismic_noise import *
import pandas as pd
import urllib.request
import json

## Paramètres initiaux

In [2]:
data_path = "DATA/apple_mobility"

## Récupération des données de mobilité

Récupération des données au moyen de la fonction `load_apple_mobility()` située dans le module `seismic_noise` à la racine du répertoire.

In [3]:
# Création et préparation des dossiers
if os.path.isdir(data_path):
    if os.path.isfile(data_path + "/apple_mobility.csv"):
        filelist = [ f for f in os.listdir(data_path) ]
        for f in filelist:
            try:
                os.remove(os.path.join(data_path, f))
            except:
                pass
else:
    os.makedirs(data_path)
    
# Téléchargement des données
data = load_apple_mobility(download_data=True)

NameError: name 'urllib' is not defined

#### Récupération des données sur la France entière

In [ ]:
data_country = load_apple_mobility(data_type = "country")
data_country

#### Récupération des données par région

In [ ]:
data_region = load_apple_mobility(data_type = "region")
data_region

#### Récupération des données par ville

In [ ]:
data_city = load_apple_mobility(data_type = "city")
data_city

## Affichage des données sur la France entière